# Reflection Pattern with LangGraph and ChatAmazonNova

This notebook demonstrates an agent that critiques and improves its own outputs through iterative reflection.

## Process Flow
1. **Generate**: Create initial content
2. **Reflect**: Critique the output
3. **Generate**: Revise based on critique
4. **Repeat**: Until approved or max iterations reached

## Key Concepts
- Self-improvement loops
- Quality iteration
- Conditional termination

In [ ]:
%env NOVA_API_KEY=<YOUR-API-KEY>
%env NOVA_BASE_URL=https://api.nova.amazon.com/v1/

In [ ]:
%pip install -e .

## Setup and Imports

In [ ]:
from typing import TypedDict, List

from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langgraph.graph import StateGraph, START, END

from langchain_amazon_nova import ChatAmazonNova

## Define State

In [ ]:
class ReflectionState(TypedDict):
    """State for the reflection loop."""
    messages: List
    iterations: int
    max_iterations: int
    initial_query: str

## Create Generator and Reflector

The generator creates content, the reflector critiques it.

In [ ]:
def create_generator(llm):
    """Create the content generator."""
    
    system_message = SystemMessage(
        content="""You are a helpful assistant that generates content based on user requests.
        If you receive critique or feedback, revise your previous response to address the concerns."""
    )
    
    def generate(state: ReflectionState) -> dict:
        messages = [system_message] + state["messages"]
        response = llm.invoke(messages)
        
        return {
            "messages": state["messages"] + [response],
            "iterations": state["iterations"] + 1,
        }
    
    return generate


def create_reflector(llm):
    """Create the content reflector/critic."""
    
    system_message = SystemMessage(
        content="""You are a thoughtful critic. Review the assistant's response and provide specific,
        constructive feedback on how it could be improved. Consider:
        - Accuracy and completeness
        - Clarity and structure
        - Tone and style appropriateness
        - Any missing important details
        
        If the response is excellent and needs no improvements, respond with exactly: "APPROVED"
        Otherwise, provide specific suggestions for improvement."""
    )
    
    def reflect(state: ReflectionState) -> dict:
        last_response = state["messages"][-1]
        
        critique_messages = [
            system_message,
            HumanMessage(content=f"Original request: {state['initial_query']}"),
            HumanMessage(content=f"Assistant's response: {last_response.content}"),
        ]
        
        response = llm.invoke(critique_messages)
        
        return {
            "messages": state["messages"] + [
                HumanMessage(content=f"Critique: {response.content}")
            ],
        }
    
    return reflect


print("Generator and reflector defined!")

## Define Routing Logic

In [ ]:
def should_continue(state: ReflectionState) -> str:
    """Decide whether to continue reflecting or end."""
    # Check if we've hit max iterations
    if state["iterations"] >= state["max_iterations"]:
        return "end"
    
    # Check if approved
    if state["messages"]:
        last_message = state["messages"][-1]
        if isinstance(last_message, HumanMessage) and "APPROVED" in last_message.content:
            return "end"
    
    # Alternate between generate and reflect
    if state["iterations"] % 2 == 1:
        return "reflect"
    else:
        return "generate"

## Build the Reflection Graph

In [ ]:
def create_reflection_graph(llm, max_iterations: int = 3):
    """Create the reflection agent graph."""
    
    generator = create_generator(llm)
    reflector = create_reflector(llm)
    
    workflow = StateGraph(ReflectionState)
    
    workflow.add_node("generate", generator)
    workflow.add_node("reflect", reflector)
    
    workflow.add_edge(START, "generate")
    
    workflow.add_conditional_edges(
        "generate",
        should_continue,
        {"reflect": "reflect", "end": END}
    )
    
    workflow.add_conditional_edges(
        "reflect",
        should_continue,
        {"generate": "generate", "end": END}
    )
    
    return workflow.compile()

## Initialize the Agent

In [ ]:
# Initialize model
llm = ChatAmazonNova(
    model="nova-pro-v1",
    temperature=0.7,
)

# Create reflection agent
max_iterations = 4
agent = create_reflection_graph(llm, max_iterations)

print(f"Reflection agent initialized with max {max_iterations} iterations!")

## Example 1: Short Essay

In [ ]:
query = "Write a short essay on the importance of renewable energy"

result = agent.invoke({
    "messages": [HumanMessage(content=query)],
    "iterations": 0,
    "max_iterations": max_iterations,
    "initial_query": query,
})

# Get final output (last AI message)
final_output = None
for msg in reversed(result["messages"]):
    if isinstance(msg, AIMessage):
        final_output = msg.content
        break

print(f"Query: {query}\n")
print("=== Final Output ===")
print(final_output)
print(f"\n(Completed in {result['iterations']} iterations)")

## Example 2: Explain a Concept

In [ ]:
query = "Explain machine learning to a 12-year-old"

result = agent.invoke({
    "messages": [HumanMessage(content=query)],
    "iterations": 0,
    "max_iterations": max_iterations,
    "initial_query": query,
})

# Get final output
final_output = None
for msg in reversed(result["messages"]):
    if isinstance(msg, AIMessage):
        final_output = msg.content
        break

print(f"Query: {query}\n")
print("=== Final Output ===")
print(final_output)
print(f"\n(Completed in {result['iterations']} iterations)")

## View Reflection Process

See the full generate-reflect-revise cycle.

In [ ]:
print(f"Total messages: {len(result['messages'])}\n")
print("Reflection process:\n")

for i, msg in enumerate(result['messages']):
    if msg.type == "human":
        if "Critique:" in msg.content:
            print(f"{i+1}. [Critique] {msg.content[:100]}...")
        else:
            print(f"{i+1}. [User] {msg.content[:100]}...")
    elif msg.type == "ai":
        print(f"{i+1}. [Generated] {msg.content[:100]}...")
    print()

## Try Your Own Query

In [ ]:
# Modify the query below
your_query = "Write a professional email declining a meeting invitation politely"

result = agent.invoke({
    "messages": [HumanMessage(content=your_query)],
    "iterations": 0,
    "max_iterations": max_iterations,
    "initial_query": your_query,
})

# Get final output
final_output = None
for msg in reversed(result["messages"]):
    if isinstance(msg, AIMessage):
        final_output = msg.content
        break

print(f"Query: {your_query}\n")
print("=== Final Output ===")
print(final_output)
print(f"\n(Completed in {result['iterations']} iterations)")